In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/tmp/satellites.csv', index_col=False)

In [3]:
del df['Name']

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
for column in df.columns:
    dtype = df.dtypes[column]
    if dtype == object:
        nans = pd.isnull(df[column])
        encoder = LabelEncoder()
        encoder.fit(df.loc[~nans,column])
        df.loc[~nans,column] = encoder.transform(df.loc[~nans,column])
        df.loc[nans,column] = float('nan')
        distargs = {'k':len(encoder.classes_)}
        distribution = 'categorical'
    else:
        distribution = 'normal'
        distargs = None
    print '%s,' % (str((distribution, distargs)))

('categorical', {'k': 79}),
('categorical', {'k': 346}),
('categorical', {'k': 18}),
('categorical', {'k': 46}),
('categorical', {'k': 4}),
('categorical', {'k': 7}),
('normal', None),
('normal', None),
('normal', None),
('normal', None),
('normal', None),
('normal', None),
('normal', None),
('normal', None),
('normal', None),
('categorical', {'k': 282}),
('categorical', {'k': 54}),
('categorical', {'k': 25}),
('categorical', {'k': 141}),
('categorical', {'k': 38}),
('normal', None),
('normal', None),


In [6]:
distributions = [
    ('categorical', {'k': 79}),
    ('categorical', {'k': 346}),
    ('categorical', {'k': 18}),
    ('categorical', {'k': 46}),
    ('categorical', {'k': 4}),
    ('categorical', {'k': 7}),
    ('normal', None),
    ('normal', None),
    ('normal', None),
    ('normal', None),
    ('normal', None),
    ('normal', None),
    ('normal', None),
    ('normal', None),
    ('normal', None),
    ('categorical', {'k': 282}),
    ('categorical', {'k': 54}),
    ('categorical', {'k': 25}),
    ('categorical', {'k': 141}),
    ('categorical', {'k': 38}),
    ('normal', None),
    ('normal', None),
]

In [7]:
import numpy as np
import sys
prng = np.random.RandomState(10)

In [8]:
from cgpm2 import sample_crosscat

In [9]:
ast = sample_crosscat.generate_random_ast(distributions, prng)
core_dsl = sample_crosscat.compile_ast_to_core_dsl(ast)
embedded_dsl = sample_crosscat.compile_core_dsl_to_embedded_dsl(core_dsl.getvalue())

In [10]:
print embedded_dsl.getvalue()

from cgpm2.categorical import Categorical
from cgpm2.crp import CRP
from cgpm2.flexible_rowmix import FlexibleRowMixture
from cgpm2.normal import Normal
from cgpm2.poisson import Poisson
from cgpm2.product import Product

view0 = FlexibleRowMixture(
  cgpm_row_divide=CRP(outputs=[100000], inputs=[], hypers={'alpha': 1.1213},),
  cgpm_components_base=Product(cgpms=[
    Categorical(outputs=[0], inputs=[], distargs={'k': 79}, hypers={'alpha': 0.7198},),
    Normal(outputs=[8], inputs=[], hypers={'s': 1.9439, 'r': 0.2218, 'm': 1.7108, 'nu': 0.4333},),
    Normal(outputs=[11], inputs=[], hypers={'s': 2.8871, 'r': 0.4847, 'm': 0.1639, 'nu': 4.3921},),])
)
view1 = FlexibleRowMixture(
  cgpm_row_divide=CRP(outputs=[100001], inputs=[], hypers={'alpha': 0.5831},),
  cgpm_components_base=Product(cgpms=[
    Categorical(outputs=[1], inputs=[], distargs={'k': 346}, hypers={'alpha': 1.051},),
    Categorical(outputs=[4], inputs=[], distargs={'k': 4}, hypers={'alpha': 0.7374},),
    Categorical(outp

In [11]:
exec(embedded_dsl.getvalue())

In [12]:
for row, values in df.iterrows():
    observation = dict(zip(range(len(values)), values.values))
    crosscat.incorporate(row, observation)

In [13]:
from cgpm2.transition_crosscat import GibbsCrossCat

In [15]:
inference = GibbsCrossCat(crosscat, prng)
inference.transition(kernels=['row_assignments'], S=10)
inference.transition(kernels=['hypers_distributions'], S=10)

Completed: 2 iterations in 11.722530 seconds.


In [17]:
inference.transition(N=1, progress=False)

In [20]:
inference.transition_structure_cpp(N=100)

Completed: 100 iterations in 30.013575 seconds.
